In [81]:
from selenium import webdriver
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import gzip
import shutil
import os
import time
import wget
import astropy
import numpy as np
from astropy.io import fits
from urllib.error import HTTPError
from bs4 import BeautifulSoup

from selenium.common.exceptions import NoSuchElementException

FORMULÁRIO DE PESQUISA

In [24]:
def HARPN(folder, PATH_DRIVER, target, fiber=None, file_type=None, download=True):
       
    #Following if's open the respective browser in silence mode and go to the TNG form website
    if PATH_DRIVER[(len(PATH_DRIVER)-12):] == 'chromedriver':
        chrome_options = webdriver.ChromeOptions()
        chrome_options.headless = True
        browser = webdriver.Chrome(options=chrome_options, executable_path= PATH_DRIVER) 
        browser.get('http://archives.ia2.inaf.it/tng/faces/search.xhtml?dswid=3397')
    if PATH_DRIVER[(len(PATH_DRIVER)-11):] == 'geckodriver':
        os.environ['MOZ_HEADLESS'] = '0'      
        browser = webdriver.Firefox(executable_path = PATH_DRIVER)
        browser.get('http://archives.ia2.inaf.it/tng/faces/search.xhtml?dswid=3397')
        time.sleep(1) #time to make sure HTML is up to date
    
    
    if download == True:
        #open single instrument panel
        browser.find_element_by_id('main:j_idt219').click()
        time.sleep(1) #time to make sure HTML is up to date  
        
        #HARPN
        browser.find_element_by_id('main:j_idt225').click()
        time.sleep(1) #time to make sure HTML is up to date

        #fiber A or B
        browser.find_element_by_id('main:FIBER_2').send_keys(fiber)
        #file_type
        browser.find_element_by_id('main:FILE_TYPE_REDUCED').send_keys(file_type) 
        
        counter=1 #in order to count the stars already processed
        for i in target: 
            #target object (using resolver and search btn)
            objectt = browser.find_element_by_name('main:j_idt104:name-resolver-text')
            objectt.send_keys( Keys.CONTROL + "a" )
            objectt.send_keys(str(i) + Keys.ENTER)                 
            WebDriverWait(browser, 20).until(EC.staleness_of(objectt))
 
            browser.find_element_by_id('main:search-btn').click()
            WebDriverWait(browser, 20).until(EC.staleness_of(browser.find_element_by_id('main:search-btn')))        

            #star identified? if yes get number of results
            HTML = BeautifulSoup(browser.page_source, 'html5lib')
            for element in HTML.find_all('input'):
                if element.get('id')=='main:j_idt104:ps_ra':
                    if element.get('value') != None:  #check if the object has values for its coordinates
                        for e in HTML.find_all('p'):
                            if str(e)[:16] == '<p>Total results':
                                num_results=(int(str(e)[17:-4]))
                                if num_results != 0: #check if the number of results are diferent than zero
                                    
                                    #hit the blue download button
                                    wait = WebDriverWait(browser, 30)
                                    wait.until(EC.element_to_be_clickable((By.ID, 'download-dropdown'))).click()
                                    time.sleep(1)

                                    #hit download button relative to .txt files
                                    browser.find_element_by_id('main:j_idt667').click()

                                    #click the files_menu
                                    files_menu = wait.until(EC.element_to_be_clickable((By.ID, 'files-menu')))
                                    time.sleep(10)
                                    files_menu.click()
                                    #identify the file name being clicked in the files menu
                                    HTML = BeautifulSoup(browser.page_source, 'html5lib')
                                    for link in HTML.find_all('a'):
                                        href = str(link.get('href'))      
                                        if 'http://archives.ia2.inaf.it/user_space/tng' in href:
                                            nome = str(href[-25:])   
                                            break

                                    #donwload the .txt in the files menu
                                    wget.download(href, folder)      

                                    #create a folder for upcoming downloads
                                    if os.path.exists(folder + str(i)):
                                        shutil.rmtree(folder + str(i))
                                    os.makedirs(folder + str(i))

                                    #open .txt downloaded file and get .gz files (wich contain the .fits files of interest)
                                    file = open(folder + '/' + str(nome))
                                    for linha in file:
                                        try:   
                                            wget.download(linha[:-1] , folder + str(i)) #last element is always garbage
                                        except HTTPError:           #in case they are private the loop continues
                                            continue    
                                    file.close()

                                    #exctrat .FITS files out of the gz files
                                    folder_i = folder + str(i)
                                    n_spectra = len(os.listdir(folder_i))
                                    for e in os.listdir(folder_i):
                                        with gzip.open(folder_i + '/' + e , 'rb') as f_in:
                                            with open( folder_i + '/' + e[:-3] , 'wb') as f_out:
                                                shutil.copyfileobj(f_in, f_out)
                                        os.remove(folder_i + '/'+ e)   #remove unnecessary .gz files                               
                                    #number of free spectra extracted
                                    
                                    print(str(counter)+')  '+ i + ': ' + str(n_spectra)+'/'+ str(num_results) + ' spectra extracted')
                                    counter +=1
                                    #SIGNAL_NOISE of downloaded spectra:
                                    Dic = {}
                                    Lista = np.zeros(69)
                                    for e in os.listdir(folder_i):
                                        ficheiro_e = fits.open( folder_i + '/' + str(e))
                                        L=[]
                                        for r in range(69):
                                            valor = (ficheiro_e[0].header['HIERARCH TNG DRS SPE EXT SN' + str(r)])**2
                                            L.append(valor)
                                        Lista = Lista+L
                                        ficheiro_e.close()
                                    for r in range(69):
                                        Dic.update({'SN' + str(r) : np.sqrt(Lista[r])})

                                    with open(folder_i + '/SIGNAL_NOISE.txt', 'w') as f:
                                        f.write(i + ': ' + str(n_spectra)+'/'+ str(num_results) + ' free spectra extracted\n')
                                        for key, value in Dic.items():
                                            f.write('%s:%f\n' % (key, value))
                                    f.close()

                                    #remove created folder for cases with 0 free spectra
                                    if n_spectra == 0:
                                        shutil.rmtree(folder_i)
                                    #remove unecessary .txt files
                                    os.remove(folder + '/'+ nome)

                                #in case the star is not in the database (but is recognized)
                                if num_results == 0:  
                                    print(str(counter)+')  '+ i + ': not in database')
                                    counter +=1

        print('DONE!')       
        #close session in the headless browser
        browser.quit()

        
        
    #performs a survey of the given stars
    if download == False:
        estrelas_nao_reconhecidas=[]
        for i in target:

            element = browser.find_element_by_id('main:j_idt104:name-resolver-text')
            element.send_keys(Keys.CONTROL + "a" )
            element.send_keys(str(i) + Keys.ENTER)
            WebDriverWait(browser, 10).until(EC.staleness_of(element))

            HTML = BeautifulSoup(browser.page_source, 'html5lib')
            for element in HTML.find_all('input'):
                if element.get('id')=='main:j_idt104:ps_ra':
                    if element.get('value') == None:
                        estrelas_nao_reconhecidas.append(i)

        browser.quit()
        k=(len(estrelas_nao_reconhecidas))
        print('Stars not recognized: ' + str(k))
        if k > 0: print(estrelas_nao_reconhecidas)
            
            
            


In [415]:
browser.quit()

In [9]:
lista_de_estrelas = ['HD1445','Corot-7','Vega','Arcturus']
HARPN('C:/Users/Henrique Legoinha/Desktop/estrelas2/' , r'C:/bin/chromedriver', lista_de_estrelas, 'A','s1d')

#LINUX
# '/home/henrique/Downloads' '/usr/local/bin/geckodriver' /home/henrique/ChromeDriver/chromedriver'
#Windows
#'C:/Users/Henrique Legoinha/Desktop/estrelas2/'    'C:/bin/geckodriver' or r'C:/bin/chromedriver'

1)  HD1445: not in database
2)  Corot-7: 1/1 spectra extracted
3)  Vega: 210/211 spectra extracted
4)  Arcturus: 0/9 spectra extracted
DONE!


In [7]:
HARPN('C:/Users/Henrique Legoinha/Desktop/estrelas2/', 'C:/bin/geckodriver', lista_de_estrelas, 'A','s1d')

1)  HD1445: not in database
2)  Corot-7: 1/1 spectra extracted
3)  Vega: 210/211 spectra extracted
4)  Arcturus: 0/9 spectra extracted
DONE!


In [5]:
HARPN('C:/Users/Henrique Legoinha/Desktop/estrelas2/' , r'C:/bin/chromedriver', lista_de_estrelas, download=False)
HARPN('C:/Users/Henrique Legoinha/Desktop/estrelas2/' , 'C:/bin/geckodriver', lista_de_estrelas, download=False)

NameError: name 'lista_de_estrelas' is not defined

In [11]:
'LISTA DE ESTRELAS DO sweet-cat'
chrome_options = webdriver.ChromeOptions()
chrome_options.headless = True
browser = webdriver.Chrome(options=chrome_options, executable_path=r'C:/bin/chromedriver') #<- !! INSERT PATH TO CHROME DRIVER
browser.get('https://www.astro.up.pt/resources/sweet-cat/')
soup = BeautifulSoup(browser.page_source, 'html5lib')
Estrelas=[]
for i in soup.find_all('a'):
    if (i.get('href')[0:13]) == 'http://simbad':
        aa= str(i.get_text())
        Estrelas.append(aa)
for i in range(len(Estrelas)):
    Estrelas[i] = Estrelas[i].replace(u'\xa0', u' ')
print(len(Estrelas))
#print(Estrelas)

# #repartir em pequenas sublistas
# lol = lambda lst, sz: [lst[i:i+sz] for i in range(0, len(lst), sz)]
# EstrelasLISTA = lol(Estrelas, 10)
#EstrelasLISTA
browser.quit()

2908


In [13]:
lista=['1SWASP J1407', '2M0805+48', '2M1059-21', '2MASS J0249-0557 (AB)', 'AD 3116', 'Cha Ha 8', 'DS Tuc A', 'EPIC 249893012', 'EPIC-206011691', 'EPIC-210490365', 'eps Ind A', 'gamma 1 Leo', 'HAT-P-10 A', 'HATS-71A', 'HIP 65A', 'IC 4651 9122', 'J1433', 'K2-293', 'K2-294', 'KELT-19 A', 'KELT-24/MASCARA-3', 'Kepler-1319 A', 'Kepler-16(AB)', 'Kepler-1647 (AB)', 'Kepler-1651 A', 'Kepler-1661 (AB)', 'Kepler-34(AB)', 'Kepler-35(AB)', 'Kepler-38(AB)', 'Kepler-413 (AB)', 'Kepler-420 A', 'Kepler-453 (AB)', 'Kepler-469', 'Kepler-47(AB)', 'Kepler-64 (AB)', 'LTT 1445A', 'MARVELS-10', 'MARVELS-11', 'MARVELS-12', 'MARVELS-13', 'MARVELS-16', 'MARVELS-17', 'MARVELS-18', 'MARVELS-19', 'MARVELS-8', 'NGTS-11', 'NGTS-3A', 'Pr 0211', 'Pr0201', 'Qatar-1', 'Qatar-10', 'Qatar-2', 'Qatar-3', 'Qatar-4', 'Qatar-5', 'Qatar-6', 'Qatar-7', 'Qatar-8', 'Qatar-9', 'SAND364', 'SDSS J080531+481233', 'SDSS J1411+2009', 'TOI-1130', 'TOI-1235', 'TOI-132', 'TOI-1338', 'TOI-1406', 'TOI-150', 'TOI-157', 'TOI-163', 'TOI-169', 'TOI-203', 'TOI-216', 'TOI-257', 'TOI-263', 'TOI-270', 'TOI-402', 'TOI-421', 'TOI-564', 'TOI-677', 'TOI-700', 'TOI-813', 'TOI-849', 'TOI-905', 'TYC-3667-1280-1', 'WASP-102', 'WASP-143', 'WASP-146', 'WASP-148', 'WASP-150', 'WASP-176', 'WASP-178/KELT-26', 'WASP-70 A', 'WASP-85 A', 'WASP-87 A', 'YBP1194', 'YBP1514', 'YBP401']
for i in lista:
    Estrelas.remove(i)

print(len(Estrelas))
print(len(lista))

2810
98


In [ ]:
#RECONHECIMENTO
HARPN('C:/Users/Henrique Legoinha/Desktop/estrelas2/' , r'C:/bin/chromedriver', Estrelas, download=False)

In [40]:
#Download

HARPN('C:/Users/Henrique Legoinha/Desktop/Estrelas/' , r'C:/bin/chromedriver', Estrelas,'A','s1d')

1)  11 Com: not in database
2)  11 UMi: not in database
3)  14 And: not in database
4)  14 Her: not in database
5)  16 Cyg B: 24/24 spectra extracted
6)  18 Del: not in database
7)  24 Boo: not in database
8)  24 Sex: 2/2 spectra extracted
9)  2MASS J15344984-2952274 A: not in database
10)  2MASS J15344984-2952274 B: not in database
11)  30 Ari B: not in database
12)  38 Vir: not in database
13)  4 Uma: 4/4 spectra extracted
14)  42 Dra: not in database
15)  47 Uma: not in database
16)  51 Peg: 434/434 spectra extracted
17)  55 Cnc: 138/138 spectra extracted
18)  6 Lyn: not in database
19)  61 Vir: not in database
20)  7 CMa: not in database
21)  70 Vir: not in database
22)  75 Cet: not in database
23)  8 Umi: not in database
24)  81 Cet: not in database
25)  91 Aqr: not in database
26)  AD Leo: 0/66 spectra extracted
27)  alf Ari: 8/8 spectra extracted
28)  AS 205 A: 6/7 spectra extracted
29)  Barnards: not in database
30)  BD+03 2562: 15/15 spectra extracted
31)  BD+14 4559: 26/26 sp

236)  HAT-P-54: 22/22 spectra extracted
237)  HAT-P-55: not in database
238)  HAT-P-56: not in database
239)  HAT-P-57: 0/87 spectra extracted
240)  HAT-P-6: 7/7 spectra extracted
241)  HAT-P-65: not in database
242)  HAT-P-66: not in database
243)  HAT-P-67: 0/50 spectra extracted
244)  HAT-P-7: 14/17 spectra extracted
245)  HAT-P-8: 8/9 spectra extracted
246)  HAT-P-9: not in database
247)  HATS-1: not in database
248)  HATS-10: not in database
249)  HATS-11: 0/1 spectra extracted
250)  HATS-12: not in database
251)  HATS-13: not in database
252)  HATS-14: not in database
253)  HATS-15: not in database
254)  HATS-16: not in database
255)  HATS-17: not in database
256)  HATS-18: not in database
257)  HATS-19: not in database
258)  HATS-2: not in database
259)  HATS-20: not in database
260)  HATS-21: not in database
261)  HATS-22: not in database
262)  HATS-23: not in database
263)  HATS-24: not in database
264)  HATS-25: not in database
265)  HATS-26: not in database
266)  HATS-27: no

484)  HD 17674: not in database
485)  HD 176986: 332/347 spectra extracted
486)  HD 177830 A: not in database
487)  HD 178911 B: not in database
488)  HD 179079: not in database
489)  HD 179949: not in database
490)  HD 180053: not in database
491)  HD 18015: not in database
492)  HD 180314: not in database
493)  HD 180777: not in database
494)  HD 180902: not in database
495)  HD 181234: not in database
496)  HD 181342: not in database
497)  HD 181433: not in database
498)  HD 181720: not in database
499)  HD 183263: not in database
500)  HD 185269: not in database
501)  HD 187085: not in database
502)  HD 187123: not in database
503)  HD 18742: not in database
504)  HD 188015: 59/60 spectra extracted
505)  HD 189567: not in database
506)  HD 189733: 188/210 spectra extracted
507)  HD 190228: 65/65 spectra extracted
508)  HD 190360: not in database
509)  HD 190647: not in database
510)  HD 190984: not in database
511)  HD 191806: not in database
512)  HD 191939: 0/5 spectra extracted


727)  HD 81688: not in database
728)  HD 81817: not in database
729)  HD 82886: not in database
730)  HD 82943: not in database
731)  HD 8326: not in database
732)  HD 83443: not in database
733)  HD 8535: not in database
734)  HD 85390: not in database
735)  HD 85512: not in database
736)  HD 85628 A: not in database
737)  HD 8574: not in database
738)  HD 86081: not in database
739)  HD 86226: not in database
740)  HD 86264: not in database
741)  HD 8673 A: not in database
742)  HD 86950: not in database
743)  HD 87646 A: not in database
744)  HD 87883: 1/2 spectra extracted
745)  HD 88133: 1/1 spectra extracted
746)  HD 89307: 30/30 spectra extracted
747)  HD 89345: 13/153 spectra extracted
748)  HD 89744: not in database
749)  HD 90156: not in database
750)  HD 9174: not in database
751)  HD 92788: not in database
752)  HD 92987: not in database
753)  HD 93083: not in database
754)  HD 93385: not in database
755)  HD 9446: not in database
756)  HD 94834: not in database
757)  HD 95

968)  Kepler-1057: 6/6 spectra extracted
969)  Kepler-1058: not in database
970)  Kepler-1059: not in database
971)  Kepler-106: 21/21 spectra extracted
972)  Kepler-1060: not in database
973)  Kepler-1061: not in database
974)  Kepler-1062: not in database
975)  Kepler-1063: not in database
976)  Kepler-1064: not in database
977)  Kepler-1065: 4/4 spectra extracted
978)  Kepler-1066: not in database
979)  Kepler-1067: not in database
980)  Kepler-1068: not in database
981)  Kepler-1069: 22/22 spectra extracted
982)  Kepler-107: 0/121 spectra extracted
983)  Kepler-1070: not in database
984)  Kepler-1071: 8/8 spectra extracted
985)  Kepler-1072: not in database
986)  Kepler-1073: not in database
987)  Kepler-1074: not in database
988)  Kepler-1075: not in database
989)  Kepler-1076: not in database
990)  Kepler-1077: not in database
991)  Kepler-1078: not in database
992)  Kepler-1079: 2/2 spectra extracted
993)  Kepler-108: not in database
994)  Kepler-1080: not in database
995)  Kepl

...

1868)  Kepler-385: not in database
1869)  Kepler-386: not in database
1870)  Kepler-387: 12/12 spectra extracted
1871)  Kepler-388: not in database
1872)  Kepler-389: not in database
1873)  Kepler-39: not in database
1874)  Kepler-390: not in database
1875)  Kepler-391: not in database
1876)  Kepler-392: not in database
1877)  Kepler-393: not in database
1878)  Kepler-394: not in database
1879)  Kepler-395: not in database
1880)  Kepler-396: not in database
1881)  Kepler-397: not in database
1882)  Kepler-398: not in database
1883)  Kepler-399: not in database
1884)  Kepler-4: 12/125 spectra extracted
1885)  Kepler-40: not in database
1886)  Kepler-400: not in database
1887)  Kepler-401: not in database
1888)  Kepler-402: 6/6 spectra extracted
1889)  Kepler-403: not in database
1890)  Kepler-404: not in database
1891)  Kepler-405: 0/48 spectra extracted
1892)  Kepler-406: not in database
1893)  Kepler-407: not in database
1894)  Kepler-408: not in database
1895)  Kepler-409: 1/1 spectr

2095)  Kepler-602: not in database
2096)  Kepler-603: not in database
2097)  Kepler-604: not in database
2098)  Kepler-605: not in database
2099)  Kepler-606: not in database
2100)  Kepler-607: not in database
2101)  Kepler-608: 1/7 spectra extracted
2102)  Kepler-609: not in database
2103)  Kepler-61: not in database
2104)  Kepler-610: not in database
2105)  Kepler-611: not in database
2106)  Kepler-612: not in database
2107)  Kepler-613: not in database
2108)  Kepler-614: not in database
2109)  Kepler-615: not in database
2110)  Kepler-616: not in database
2111)  Kepler-617: not in database
2112)  Kepler-618: not in database
2113)  Kepler-619: 4/4 spectra extracted
2114)  Kepler-62: not in database
2115)  Kepler-620: 4/4 spectra extracted
2116)  Kepler-621: not in database
2117)  Kepler-622: not in database
2118)  Kepler-623: not in database
2119)  Kepler-624: not in database
2120)  Kepler-625: not in database
2121)  Kepler-626: not in database
2122)  Kepler-627: not in database
2123

2323)  Kepler-811: not in database
2324)  Kepler-812: not in database
2325)  Kepler-813: not in database
2326)  Kepler-814: not in database
2327)  Kepler-815: not in database
2328)  Kepler-816: not in database
2329)  Kepler-817: not in database
2330)  Kepler-818: not in database
2331)  Kepler-819: not in database
2332)  Kepler-82: not in database
2333)  Kepler-820: not in database
2334)  Kepler-821: not in database
2335)  Kepler-822: not in database
2336)  Kepler-823: 4/4 spectra extracted
2337)  Kepler-824: not in database
2338)  Kepler-825: not in database
2339)  Kepler-826: 4/4 spectra extracted
2340)  Kepler-827: not in database
2341)  Kepler-828: not in database
2342)  Kepler-829: not in database
2343)  Kepler-83: not in database
2344)  Kepler-830: not in database
2345)  Kepler-831: not in database
2346)  Kepler-832: not in database
2347)  Kepler-833: not in database
2348)  Kepler-834: not in database
2349)  Kepler-835: not in database
2350)  Kepler-836: not in database
2351)  Kep

2551)  KOI-415: not in database
2552)  KOI-4427: not in database
2553)  KOI-5833: not in database
2554)  KOI-5929: not in database
2555)  KOI-682: not in database
2556)  KOI-771: not in database
2557)  KOI-830: not in database
2558)  KPS-1: not in database
2559)  ksi Aql: 12/12 spectra extracted
2560)  L 168-9: not in database
2561)  L 98-59: not in database
2562)  Lalande 21185: 102/102 spectra extracted
2563)  LHS 1140: not in database
2564)  LHS 1815: not in database
2565)  LHS 3844: not in database
2566)  LP 358-499: not in database
2567)  LP 791-18: not in database
2568)  LP415-17: 6/59 spectra extracted
2569)  LSPM J2116+0234: not in database
2570)  LTT 3780: 0/31 spectra extracted
2571)  MASCARA-1: 101/230 spectra extracted
2572)  mu Ara: not in database
2573)  mu Leo: not in database
2574)  NGC 2423 3: not in database
2575)  NGC 4349 127: not in database
2576)  NGTS-1: not in database
2577)  NGTS-10: not in database
2578)  NGTS-2: not in database
2579)  NGTS-4: not in database


2788)  WASP-94 B: not in database
2789)  WASP-95: not in database
2790)  WASP-96: not in database
2791)  WASP-97: not in database
2792)  WASP-98: not in database
2793)  WASP-99: not in database
2794)  WD 0137-349: not in database
2795)  WD 1145+017: 0/74 spectra extracted
2796)  Wolf 1061: not in database
2797)  Wolf 359: 0/38 spectra extracted
2798)  Wolf 503: 0/20 spectra extracted
2799)  WTS-1: not in database
2800)  WTS-2: not in database
2801)  XO-1: 11/11 spectra extracted
2802)  XO-2N: 135/178 spectra extracted
2803)  XO-2S: 135/178 spectra extracted
2804)  XO-3: 65/66 spectra extracted
2805)  XO-4: 5/5 spectra extracted
2806)  XO-5: not in database
2807)  XO-6: not in database
2808)  XO-7: not in database
2809)  YZ Cet: not in database
DONE!


In [156]:
'COORDENADAS DAS ESTRELAS NAO RECONHECIDAS'

# 'LISTA DE ESTRELAS DO sweet-cat'
# chrome_options = webdriver.ChromeOptions()
# chrome_options.headless = True
# browser = webdriver.Chrome(options=chrome_options, executable_path=r'C:/bin/chromedriver') #<- !! INSERT PATH TO CHROME DRIVER
# browser.get('https://www.astro.up.pt/resources/sweet-cat/')
# soup = BeautifulSoup(browser.page_source, 'html5lib')
lista = ['1SWASP J1407', '2M0805+48', '2M1059-21', '2MASS J0249-0557 (AB)', 'AD 3116', 'Cha Ha 8', 'DS Tuc A', 'EPIC 249893012', 'EPIC-206011691', 'EPIC-210490365', 'eps Ind A', 'gamma 1 Leo', 'HAT-P-10 A', 'HATS-71A', 'HIP 65A', 'IC 4651 9122', 'J1433', 'K2-293', 'K2-294', 'KELT-19 A', 'KELT-24/MASCARA-3', 'Kepler-1319 A', 'Kepler-16(AB)', 'Kepler-1647 (AB)', 'Kepler-1651 A', 'Kepler-1661 (AB)', 'Kepler-34(AB)', 'Kepler-35(AB)', 'Kepler-38(AB)', 'Kepler-413 (AB)', 'Kepler-420 A', 'Kepler-453 (AB)', 'Kepler-469', 'Kepler-47(AB)', 'Kepler-64 (AB)', 'LTT 1445A', 'MARVELS-10', 'MARVELS-11', 'MARVELS-12', 'MARVELS-13', 'MARVELS-16', 'MARVELS-17', 'MARVELS-18', 'MARVELS-19', 'MARVELS-8', 'NGTS-11', 'NGTS-3A', 'Pr 0211', 'Pr0201', 'Qatar-1', 'Qatar-10', 'Qatar-2', 'Qatar-3', 'Qatar-4', 'Qatar-5', 'Qatar-6', 'Qatar-7', 'Qatar-8', 'Qatar-9', 'SAND364', 'SDSS J080531+481233', 'SDSS J1411+2009', 'TOI-1130', 'TOI-1235', 'TOI-132', 'TOI-1338', 'TOI-1406', 'TOI-150', 'TOI-157', 'TOI-163', 'TOI-169', 'TOI-203', 'TOI-216', 'TOI-257', 'TOI-263', 'TOI-270', 'TOI-402', 'TOI-421', 'TOI-564', 'TOI-677', 'TOI-700', 'TOI-813', 'TOI-849', 'TOI-905', 'TYC-3667-1280-1', 'WASP-102', 'WASP-143', 'WASP-146', 'WASP-148', 'WASP-150', 'WASP-176', 'WASP-178/KELT-26', 'WASP-70 A', 'WASP-85 A', 'WASP-87 A', 'YBP1194', 'YBP1514', 'YBP401']

RA=[]
Dec=[]

for i in range(2908):
    estupidez=[]
    
    est = (soup.find_all('tr')[i+4]).find_all('td')[0].get_text()
    for l in range(len(est)):
        estupidez.append(est[l])

    for r in estupidez:
        if r == '\xa0': estupidez.remove(r)
    
    super_estupido=[]
    for e in lista:
        ainda_mais_estupido=[]
        for le in range(len(e)):
            ainda_mais_estupido.append(e[le])
        for r in ainda_mais_estupido:
            if r == ' ': ainda_mais_estupido.remove(r)

        if estupidez == ainda_mais_estupido:
            RA.append(soup.find_all('tr')[i+4].find_all('td')[2].get_text())
            Dec.append(soup.find_all('tr')[i+4].find_all('td')[3].get_text())


RD=[]
for i in range(len(RA)):
    RD.append(str(RA[i])+ ' ' + str(Dec[i]))
for i in RD:
    print(i)

14 07 48.00 -39 45 43.00
08 05 32.00 +40 12 33.00
10 59 51.42 -21 13 08.58
02 49 56.38 -05 57 35.41
08 42 39.44 +19 24 51.89
11 07 46.09 -77 40 08.91
23 39 39.48 -69 11 44.70
15 12 59.56 -16 43 28.19
22 41 12.89 -14 29 20.35
04 13 06.00 +15 14 52.00
22 03 22.00 -56 47 10.00
10 19 58.35 +19 50 29.34
03 09 28.54 +30 40 24.73
01 02 12.00 -61 45 22.00
00 00 44.90 -54 49 49.85
17 24 50.07 -49 56 56.08
14 33 18.00 +10 11 23.00
23 26 26.00 -6 01 40.00
23 28 12.00 -6 01 40.04
07 26 03.00 +07 36 57.00
10 47 38.35 +71 39 21.16
19 23 49.46 +49 21 58.7
19 16 18.17 +51 45 26.76
19 52 36.00 +40 39 22.00
18 54 30.80 +48 23 27.75
19 00 40.11 +41 58 01.38
19 45 44.59 +44 38 29.62
19 37 59.27 +46 41 23.14
19 07 19.29 +42 16 45.14
19 14 02.55 +51 09 44.90
19 24 54.04 +44 55 38.58
18 52 57.91 +46 22 42.50
19 49 54.21 +41 06 51.44
19 41 11.50 +46 55 13.68
19 52 51.63 +39 57 18.36
03 01 51.39 -16 35 36.10
12 42 38.61 -07 12 57.35
13 11 29.98 +15 43 18.97
13 33 35.95 +52 26 13.38
13 45 20.55 +48 26 40.63
18 

In [382]:
'SIMBAD'
chrome_options = webdriver.ChromeOptions()
chrome_options.headless = True
browser = webdriver.Chrome(options=chrome_options, executable_path=r'C:/bin/chromedriver') #<- !! INSERT PATH TO CHROME DRIVER
browser.get('http://simbad.u-strasbg.fr/simbad/sim-fcoo')
# soup = BeautifulSoup(browser.page_source, 'html5lib')


In [405]:
star_names = []
unrec = []

for i in range(len(RD)):
    input_coordinates = browser.find_element_by_name('Coord')
    input_coordinates.send_keys(Keys.CONTROL + "a")
    input_coordinates.send_keys(RD[i] + Keys.ENTER)        
    
    HTML = BeautifulSoup(browser.page_source, 'html5lib')
    not_found = str(HTML.find_all('b')[-1])
    
    if not_found == '<b>No astronomical object found :</b>':
        unrec.append(RD[i])
        print(RD[i])
        browser.get('http://simbad.u-strasbg.fr/simbad/sim-fcoo')
        
    if not_found != '<b>No astronomical object found :</b>':
        try:
            for h in HTML.find_all('td'):
                if h.get('valign') == 'MIDDLE':
                    size=int((h.text)[17:])
                    break
        
            HTML = BeautifulSoup(browser.page_source, 'html5lib')
            j = HTML.find_all('tr')
            enum=0
            for b in range(size):
                sp_type = str((j[b+4].find_all('td')[11]))[5:-10]
                enum +=1
                if sp_type != '~' :
                    star_names.append(j[b+4].find_all('td')[1].text[1:])
                    break
                
                if enum == size:
                    unrec.append(RD[i])
                
        except ValueError : 
            HTML = BeautifulSoup(browser.page_source, 'html5lib')
            star_names.append(str(HTML.find_all('b')[4])[3:-4])
            print(str(HTML.find_all('b')[4])[3:-4])
            browser.get('http://simbad.u-strasbg.fr/simbad/sim-fcoo')
            
for i in range(len(star_names)):
    star_names[i] = star_names[i].replace('\n', '')

0
ASAS J140748-3945.7    
1
08 05 32.00 +40 12 33.00
2

2MASS J10595138-2113082
        
3
2MASS J02495639-0557352    
4
Cl* NGC 2632   HSHJ     430    
5
Ass Cha T 1-15    
6
HD 222259    
7
ENUM 5
15 12 59.56 -16 43 28.19
8
K2-21    
9
K2-25    
10
* eps Ind    
11
* gam Leo    
12
WASP-11    
13
BPS CS 22953-0003    
14
ENUM 2
00 00 44.90 -54 49 49.85
15
IC 4651   29    
16
ENUM 3
14 33 18.00 +10 11 23.00
17
ENUM 2
23 26 26.00 -6 01 40.00
18

TYC 5250-220-1
        
19
BD+07  1721    
20
HD  93148    
21
Kepler-1319    
22
Kepler-16    
23
V* V1296 Cyg    
24
Kepler-1651    
25
2MASS J19003180+4158269    
26
ENUM 8
19 45 44.59 +44 38 29.62
27
KIC 9837573    
28
2MASS J19071472+4216201    
29
ENUM 4
19 14 02.55 +51 09 44.90
30
Kepler-420    
31
Kepler-453    
32
KIC 5897901    
33
ENUM 8
19 41 11.50 +46 55 13.68
34
2MASS J19524819+3957170    
35
BD-17   588    
36

BD-06  3628
        
37
HD 114623    
38

TYC 3469-492-1
        
39
SDSS J134519.76+482813.5    
40
ENUM 5
18 43 07.92 

In [412]:
len(star_names)
# for i in range(len(unrec)):
#     print(unrec[i])


64